In [1]:
import muon
import scanpy as sc
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy

In [18]:
###---------------------Load data

madata = muon.read("multiome_pilot_unfilt.h5mu")
df_samples = pd.read_csv('/nemo/project/proj-gandhis-asap/multiome_cellranger_arc/Samples_NewGenome/Multiome_finished/scripts_cellranger/data/cellranger_arc_aggr.csv')
df = pd.read_csv("ASAP_wood_project_selected_demogrpahics.csv")
batches = pd.read_csv('../01-QC/Wood_batches_pilot_edited.csv')

In [19]:
###---------------------Prepare annotation
C = copy.copy(df)
P = copy.copy(df)
F = copy.copy(df)
C['Case'] = C['Case']+'C'
C['region'] = 'C'
P['Case'] = P['Case']+'P'
P['region'] = 'P'
F['Case'] = F['Case']+'F'
F['region'] = 'F'
kit5 = P[P['Case']=='C046P']
kit5['Case'] = kit5['Case']+'_kit5'

###---------------------Merge sample_id
madata['rna'].obs.reset_index(inplace = True)
madata['rna'].obs['new'] = madata['rna'].obs['index'].astype(str)
madata['rna'].obs['new'] = (madata['rna'].obs['new'].str.split('-', expand = True)[1]).astype(str)
df_samples['index'] = pd.DataFrame(df_samples['library_id']).astype(str).reset_index()['index'].astype(int)
df_samples['index'] = (df_samples['index']+1).astype(str)
merged = madata['rna'].obs.merge(df_samples, left_on = 'new', right_on = 'index', how = 'left')
madata['rna'].obs['sample_id'] = list(merged['library_id'])
madata['rna'].obs['sample_id']

madata['rna'].obs['sample_id'] = madata['rna'].obs['sample_id'].astype(str)
df['Case'] = df['Case'].astype(str)

###---------------------Merge annotation
all_annot = pd.concat([C,P,F, kit5], axis = 0)
all_annot = all_annot.merge(batches, left_on = 'Case', right_on = 'sample_id', how = 'left')
all_annot.to_csv('list_55samples_pilot_withmetadata.csv')

/tmp/slurm_59360175/ipykernel_274509/2043008382.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kit5['Case'] = kit5['Case']+'_kit5'


In [4]:
madata['rna'].obs = madata['rna'].obs.merge(all_annot[['Case','Age', 'Sex', 'Onset', 'Duration', 'Dementia', 'PMI',
       'SH/X', 'FixFr', 'SC', 'CSF', 'Q.', 'Vol.', 'RIN', 'Weight', 'NPD',
        'aSN', 'Tau', 'aB', 'Group', 'Fresh/Fix',
       'Suitable', 'Pathology', 'region', 'batch']], left_on = 'sample_id', right_on = 'Case', how = 'left')
#madata['rna'].obs = madata['rna'].obs.merge(batches, left_on = 'sample_id', right_on = 'sample_id', how = 'left')
madata['rna'].obs.set_index('index', inplace = True)
madata['rna'].obs.columns

/camp/lab/gandhis/home/users/grantpm/.conda/envs/panpipes/lib/python3.8/site-packages/anndata/_core/anndata.py:798: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


Index(['sample_id', 'doublet_scores', 'predicted_doublets',
       'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts',
       'log1p_total_counts', 'total_counts_hb', 'log1p_total_counts_hb',
       'pct_counts_hb', 'total_counts_mt', 'log1p_total_counts_mt',
       'pct_counts_mt', 'total_counts_rp', 'log1p_total_counts_rp',
       'pct_counts_rp', 'total_counts_ig', 'log1p_total_counts_ig',
       'pct_counts_ig', 'MarkersNeutro_score', 'S_score', 'G2M_score', 'phase',
       'new', 'Case', 'Age', 'Sex', 'Onset', 'Duration', 'Dementia', 'PMI',
       'SH/X', 'FixFr', 'SC', 'CSF', 'Q.', 'Vol.', 'RIN', 'Weight', 'NPD',
       'aSN', 'Tau', 'aB', 'Group', 'Fresh/Fix', 'Suitable', 'Pathology',
       'region', 'batch'],
      dtype='object')

In [5]:
madata.obs.drop(columns = 'sample_id', inplace = True)
madata.obs = madata.obs.merge(madata['rna'].obs[['sample_id','Age', 'Sex', 'Onset', 'Duration', 'Dementia', 'PMI',
       'SH/X', 'FixFr', 'SC', 'CSF', 'Q.', 'Vol.', 'RIN', 'Weight', 'NPD',
        'aSN', 'Tau', 'aB', 'Group', 'Fresh/Fix',
       'Suitable', 'Pathology', 'region', 'batch']], right_index = True, left_index = True, how = 'left')


In [6]:
madata['atac'].obs.drop(columns = 'sample_id', inplace = True)
madata['atac'].obs = madata['atac'].obs.merge(madata['rna'].obs[['sample_id','Age', 'Sex', 'Onset', 'Duration', 'Dementia', 'PMI',
       'SH/X', 'FixFr', 'SC', 'CSF', 'Q.', 'Vol.', 'RIN', 'Weight', 'NPD',
        'aSN', 'Tau', 'aB', 'Group', 'Fresh/Fix',
       'Suitable', 'Pathology', 'region', 'batch']], right_index = True, left_index = True, how = 'left')


In [7]:
madata.write('annotated_multiome_pilot_unfilt.h5mu')

/camp/lab/gandhis/home/users/grantpm/.conda/envs/panpipes/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c
/camp/lab/gandhis/home/users/grantpm/.conda/envs/panpipes/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c


In [ ]:
madata = muon.read("annotated_multiome_pilot_unfilt.h5mu")

madata['rna'].obs

In [9]:
madata['rna'].obs.columns

Index(['sample_id', 'doublet_scores', 'predicted_doublets',
       'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts',
       'log1p_total_counts', 'total_counts_hb', 'log1p_total_counts_hb',
       'pct_counts_hb', 'total_counts_mt', 'log1p_total_counts_mt',
       'pct_counts_mt', 'total_counts_rp', 'log1p_total_counts_rp',
       'pct_counts_rp', 'total_counts_ig', 'log1p_total_counts_ig',
       'pct_counts_ig', 'MarkersNeutro_score', 'S_score', 'G2M_score', 'phase',
       'new', 'Case', 'Age', 'Sex', 'Onset', 'Duration', 'Dementia', 'PMI',
       'SH/X', 'FixFr', 'SC', 'CSF', 'Q.', 'Vol.', 'RIN', 'Weight', 'NPD',
       'aSN', 'Tau', 'aB', 'Group', 'Fresh/Fix', 'Suitable', 'Pathology',
       'region', 'batch'],
      dtype='object')

In [16]:
pd.DataFrame(madata['rna'].obs['sample_id'].unique()).to_csv('list_55samples_pilot.csv')

In [26]:
all_annot.set_index('Case').loc[madata['rna'].obs['sample_id'].unique()].to_csv('list_55samples_pilot_withmetadata.csv')

In [23]:
madata['rna'].obs['sample_id'].unique()

array(['PD0086P', 'PD0980F', 'PD0945P', 'C073P', 'PDC034C', 'PDC029F',
       'PD0109F', 'PD0945C', 'PDC087F', 'C030C', 'PD0687F', 'PD0022F',
       'C074P', 'PD0086F', 'PD0913P', 'PDC034F', 'PD0590C', 'C043C',
       'PDC085P', 'PD0109C', 'C076P', 'C073F', 'PD0822C', 'PD0980P',
       'PD0822F', 'PDC035C', 'PD0779F', 'C074F', 'PDC029C', 'C076C',
       'PD0969F', 'PD0596P', 'C073C', 'C075P', 'PD0109P', 'PD0913F',
       'PD0086C', 'PDC085C', 'C046C', 'PD0779C', 'C030F', 'PDC085F',
       'PDC035F', 'C075F', 'PDC034P', 'PDC035P', 'C046P_kit5', 'PDC022F',
       'C076F', 'PD0612F', 'PD0612C', 'PD0612P', 'PD0822P', 'PD0969C',
       'PD0022C'], dtype=object)